In [ ]:
from google.colab import drive
import torch
from torch import nn
from torch.optim import Adam
from torchvision import datasets, transforms, models
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torchvision.models import vgg19
from torch.utils.data import Subset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from google.colab import drive
import torch.nn.functional as F

# Mount Google Drive
drive.mount('/content/drive')

# Define dataset transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/test2', transform=transform)

# Prepare 3-fold cross-validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
fold_indices = list(kfold.split(dataset))


Mounted at /content/drive


In [ ]:

# Grey Wolf Optimizer (GWO) Class
class GreyWolfOptimizer:
    def __init__(self, objective_function, dim, lb, ub, population_size, max_iter):
        self.obj_function = objective_function
        self.dim = dim  # Dimensionality (1 for learning rate)
        self.lb = lb    # Lower bound
        self.ub = ub    # Upper bound
        self.pop_size = population_size
        self.max_iter = max_iter
        self.alpha_score = float("inf")
        self.beta_score = float("inf")
        self.delta_score = float("inf")
        self.alpha_pos = None
        self.beta_pos = None
        self.delta_pos = None
        self.positions = np.random.uniform(lb, ub, (population_size, dim))

    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.pop_size):
                fitness = self.obj_function(self.positions[i, :])

                # Update Alpha, Beta, Delta wolves
                if fitness < self.alpha_score:
                    self.alpha_score, self.alpha_pos = fitness, self.positions[i, :].copy()
                elif fitness < self.beta_score:
                    self.beta_score, self.beta_pos = fitness, self.positions[i, :].copy()
                elif fitness < self.delta_score:
                    self.delta_score, self.delta_pos = fitness, self.positions[i, :].copy()

            # Update positions
            for i in range(self.pop_size):
                for j in range(self.dim):
                    r1, r2 = np.random.rand(), np.random.rand()
                    A1 = 2 * r1 - 1
                    C1 = 2 * r2
                    D_alpha = abs(C1 * self.alpha_pos[j] - self.positions[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha

                    r1, r2 = np.random.rand(), np.random.rand()
                    A2 = 2 * r1 - 1
                    C2 = 2 * r2
                    D_beta = abs(C2 * self.beta_pos[j] - self.positions[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta

                    r1, r2 = np.random.rand(), np.random.rand()
                    A3 = 2 * r1 - 1
                    C3 = 2 * r2
                    D_delta = abs(C3 * self.delta_pos[j] - self.positions[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta

                    self.positions[i, j] = np.clip((X1 + X2 + X3) / 3, self.lb, self.ub)

        return self.alpha_pos, self.alpha_score




In [ ]:
def objective_function(lr):
    # Load VGG19 pretrained model
    model = vgg19(pretrained=True)
    model.classifier[6] = torch.nn.Linear(4096, len(dataset.classes))  # Adjust for number of classes
    model = model.to('cuda')

    # Create optimizer with current learning rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr[0])
    criterion = torch.nn.CrossEntropyLoss()

    # Use the first fold for training and validation
    train_idx, val_idx = fold_indices[0]
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

    # Training loop
    model.train()
    for epoch in range(3):  # Example: Train for 3 epochs
        for inputs, labels in train_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Validate and calculate accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return -accuracy  # GWO minimizes the objective, so return negative accuracy



In [ ]:

# Initialize Grey Wolf Optimizer to find the best learning rate
gwo = GreyWolfOptimizer(
    objective_function=objective_function,
    dim=1,               # One-dimensional optimization (learning rate)
    lb=1e-5,             # Lower bound of learning rate
    ub=1e-1,             # Upper bound of learning rate
    population_size=10,  # Population size
    max_iter=20          # Max iterations
)

In [ ]:

# Run GWO to find optimal learning rate
optimal_lr, best_fitness = gwo.optimize()
print(f"Optimal Learning Rate: {optimal_lr[0]}")
print(f"Best Validation Accuracy: {-best_fitness}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:10<00:00, 53.5MB/s]


Optimal Learning Rate: 8.377642111712619e-05
Best Validation Accuracy: 0.7567567567567568


In [ ]:
# Now, train the final model using the optimal learning rate
final_lr = optimal_lr[0]
model = vgg19(pretrained=True)
model.classifier[6] = torch.nn.Linear(4096, len(dataset.classes))  # Adjust for the number of classes
model = model.to('cuda')

In [ ]:
# Train using optimal learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=final_lr)
criterion = torch.nn.CrossEntropyLoss()

# Use the first fold again for final training
train_idx, val_idx = fold_indices[0]
train_subset = Subset(dataset, train_idx)
val_subset = Subset(dataset, val_idx)
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)


In [ ]:

# Final training loop
for epoch in range(3):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Function to train and evaluate the model for a given fold
def evaluate_fold(train_idx, val_idx, learning_rate):
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

    # Define the model
    model = vgg19(pretrained=True)
    model.classifier[6] = torch.nn.Linear(4096, len(dataset.classes))  # Adjust for number of classes
    model = model.to('cuda')

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    # Training loop
    model.train()
    for epoch in range(3):  # Example: Train for 3 epochs
        for inputs, labels in train_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Validation phase
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # Metrics calculation
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_true, y_pred)

    return accuracy, precision, recall, f1, conf_matrix



In [ ]:
# Perform cross-validation for each fold
fold_results = []
for fold, (train_idx, val_idx) in enumerate(fold_indices):
    print(f"Fold {fold + 1}:")
    accuracy, precision, recall, f1, conf_matrix = evaluate_fold(train_idx, val_idx, optimal_lr[0])
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}\n")
    fold_results.append((accuracy, precision, recall, f1, conf_matrix))


Fold 1:


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Accuracy: 0.6486
Precision: 0.6514
Recall: 0.6486
F1 Score: 0.6481
Confusion Matrix:
[[25 11]
 [15 23]]

Fold 2:


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Accuracy: 0.7123
Precision: 0.7126
Recall: 0.7123
F1 Score: 0.7123
Confusion Matrix:
[[26 10]
 [11 26]]

Fold 3:


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Accuracy: 0.7671
Precision: 0.7913
Recall: 0.7671
F1 Score: 0.7601
Confusion Matrix:
[[35  3]
 [14 21]]



In [ ]:
# Print average metrics across all folds
avg_accuracy = np.mean([result[0] for result in fold_results])
avg_precision = np.mean([result[1] for result in fold_results])
avg_recall = np.mean([result[2] for result in fold_results])
avg_f1 = np.mean([result[3] for result in fold_results])

print("Average Metrics Across All Folds:")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")

Average Metrics Across All Folds:
Average Accuracy: 0.7094
Average Precision: 0.7185
Average Recall: 0.7094
Average F1 Score: 0.7069


In [ ]:

# Final evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


In [ ]:
# Performance metrics
conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=dataset.classes)
accuracy = accuracy_score(y_true, y_pred)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{report}")
print(f"Accuracy: {accuracy}")

Confusion Matrix:
[[17 19]
 [ 7 31]]
Classification Report:
              precision    recall  f1-score   support

        Fake       0.71      0.47      0.57        36
        Real       0.62      0.82      0.70        38

    accuracy                           0.65        74
   macro avg       0.66      0.64      0.64        74
weighted avg       0.66      0.65      0.64        74

Accuracy: 0.6486486486486487
